# Ugesco : enrich and desambiguate a sample of the Samnang-Hans' JSON

In [1]:
from ugesco import *

D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, fa

In [2]:
#test sur un Json contenant 20 éléments pris au hasard
with open(r"C:\Users\ettor\Documents\GitHub\UGESCO\data\jsondata_ugesco.json", "r", encoding="utf8") as file:
    json = json.load(file)
    
sample = random.sample(json, k=20)


In [3]:
data = pd.DataFrame([flatten(line) for line in sample])

In [4]:
data.head()

,beeldid,imageclassification_class1,imageclassification_class2,imageclassification_class3,imageclassification_class4,imageclassification_class5,imageclassification_prob1,imageclassification_prob2,imageclassification_prob3,imageclassification_prob4,...,spatial_key_3,spatial_value_0,spatial_value_1,spatial_value_2,spatial_value_3,temporal,temporal_key_0,temporal_key_1,temporal_value_0,temporal_value_1
0,00035473,street,crosswalk,gas_station,phone_booth,bus_station/indoor,0.463978,0.399446,0.064499,0.019478,...,NaN,NaN,NaN,NaN,NaN,NaN,TIMEX_MISC,TIMEX_DATE,la semaine de la circulation de Bruxelles,3/4/1936
1,00015666,plaza,street,courtyard,fire_escape,auto_factory,0.269459,0.092110,0.088286,0.056132,...,LOC_PLACE,la Grande Place,Anvers,la Grand'Place,Anvers,NaN,TIMEX_MISC,NaN,40,NaN
2,00002751,plaza,ice_skating_rink/outdoor,archive,street,army_base,0.477765,0.106013,0.097137,0.053500,...,NaN,NaN,NaN,NaN,NaN,NaN,TIMEX_DATE,NaN,24 mai 1940,NaN
3,00009589,palace,plaza,apartment_building/outdoor,office_building,tower,0.320755,0.185312,0.112493,0.072328,...,NaN,Maison du roi,Bruxelles,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00012338,church/outdoor,palace,castle,slum,plaza,0.152554,0.124778,0.097434,0.085449,...,NaN,Bruxelles,la Grand'Place,NaN,NaN,NaN,TIMEX_MISC,TIMEX_DATE,La fête de Chants Flamands,31 aôut 1941


In [5]:
# exclude image classifications and temporal values for the moment
data = data[data.columns.drop(list(data.filter(regex='imageclassification|temporal')))]

In [6]:
# reshape the dataframe verticaly
data = data.set_index('beeldid')
data.columns = data.columns.str.split('_', expand=True)

In [7]:
data = data.stack().reset_index(level=1, drop=True).reset_index()

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2910: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)
D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2910: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
data.columns = ['_'.join(col) for col in data.columns]

In [9]:
data.rename(columns={'beeldid_': 'beeldid'}, inplace=True)

In [10]:
data.head()

,beeldid,spatial_value,spatial_key,spatial_value
0,00015666,Anvers,LOC_PLACE,Anvers
1,00015666,la Grande Place,LOC_STREET,la Grande Place
2,00015666,Anvers,LOC_PLACE,Anvers
3,00015666,la Grand'Place,LOC_STREET,la Grand'Place
4,00015666,Anvers,LOC_PLACE,Anvers


In [11]:
#join/merge with phototheque_pallas to get the locations. 
#The csv is gzipped to get around the 100MB file weigth limitation of Github
phototeque = pd.read_csv("data/phototheque_pallas.csv.gz", compression="gzip", encoding="utf8", dtype={'beeldid': str, 'trfwnumm': str})

data = pd.merge(data, phototeque, how='left', on=['beeldid'], suffixes=['', '_x'])

In [12]:
# Pictures that contains more than a loc in the thesaurus descriptors

#data[data['loc_qid'].str.contains(",", na = False)]

In [13]:
data.head()

,beeldid,spatial_value,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name
0,00015666,Anvers,LOC_PLACE,Anvers,http://pallas.cegesoma.be/pls/opac/opac.search...,Remise officielle de 40 drapeaux à la DM-ZB : ...,Marche organisée par la Milice thioise-Brigade...,750,NaN,Q31,BE,Belgique,Q12892,Anvers
1,00015666,Anvers,LOC_PLACE,Anvers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q234,Flandre
2,00015666,la Grande Place,LOC_STREET,la Grande Place,http://pallas.cegesoma.be/pls/opac/opac.search...,Remise officielle de 40 drapeaux à la DM-ZB : ...,Marche organisée par la Milice thioise-Brigade...,750,NaN,Q31,BE,Belgique,Q12892,Anvers
3,00015666,la Grande Place,LOC_STREET,la Grande Place,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q234,Flandre
4,00015666,Anvers,LOC_PLACE,Anvers,http://pallas.cegesoma.be/pls/opac/opac.search...,Remise officielle de 40 drapeaux à la DM-ZB : ...,Marche organisée par la Milice thioise-Brigade...,750,NaN,Q31,BE,Belgique,Q12892,Anvers


In [14]:
# Merge with NER_classes (spatial_keys matched with wikidata)
ner_classes = pd.read_csv("data/ner_classes.csv")

data = pd.merge(data, ner_classes, how='left', left_on = ['spatial_key'], right_on = ['ner_class'])

In [15]:
# rename the new columns merged
data.rename(columns={'wiki_qid': 'ner_class_qid', 'wiki_class':'ner_class_name'}, inplace=True)

In [16]:
#remove duplicate columns
data = data.T.drop_duplicates().T

In [17]:
data.head()

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00015666,Anvers,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Remise officielle de 40 drapeaux à la DM-ZB : ...,Marche organisée par la Milice thioise-Brigade...,750,NaN,Q31,BE,Belgique,Q12892,Anvers,Q56061,administrative territorial entity
1,00015666,Anvers,LOC_PLACE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q234,Flandre,Q56061,administrative territorial entity
2,00015666,la Grande Place,LOC_STREET,http://pallas.cegesoma.be/pls/opac/opac.search...,Remise officielle de 40 drapeaux à la DM-ZB : ...,Marche organisée par la Milice thioise-Brigade...,750,NaN,Q31,BE,Belgique,Q12892,Anvers,Q83620,thoroughfare
3,00015666,la Grande Place,LOC_STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q234,Flandre,Q83620,thoroughfare
4,00015666,Anvers,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Remise officielle de 40 drapeaux à la DM-ZB : ...,Marche organisée par la Milice thioise-Brigade...,750,NaN,Q31,BE,Belgique,Q12892,Anvers,Q56061,administrative territorial entity


In [18]:
# remove articles in spatial_value
data['spatial_value'] = data['spatial_value'].str.replace(pat=r"\b(le|la|l\s+'|l'|les)\b", repl="", n=1, case=False)

In [19]:
#data.to_csv(r"C:\Users\ettor\Desktop\data_ugesco.csv", header=True, index=False, encoding="utf-8")

In [20]:
#normalize spatial_value
data['spatial_value'] = data['spatial_value'].apply(lambda x: Fingerprinter(x).get_fingerprint_nonsorted())

In [21]:
#transformer les loc_place pays en loc_country
with open(r"C:\Users\ettor\Documents\GitHub\UGESCO\data\countries.csv", 'r', encoding="utf8") as f:
    countries = set([Fingerprinter(country).get_fingerprint_nonsorted() for country in f])

# Grab DataFrame rows where column value is present in a list
data.loc[(data['spatial_value'].isin(countries)), ['spatial_key']] = "LOC_COUNTRY"

In [22]:
gb = data.groupby(['beeldid'])

In [23]:
gb.head() 

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00015666,anvers,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Remise officielle de 40 drapeaux à la DM-ZB : ...,Marche organisée par la Milice thioise-Brigade...,750,NaN,Q31,BE,Belgique,Q12892,Anvers,Q56061,administrative territorial entity
1,00015666,anvers,LOC_PLACE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q234,Flandre,Q56061,administrative territorial entity
2,00015666,grande place,LOC_STREET,http://pallas.cegesoma.be/pls/opac/opac.search...,Remise officielle de 40 drapeaux à la DM-ZB : ...,Marche organisée par la Milice thioise-Brigade...,750,NaN,Q31,BE,Belgique,Q12892,Anvers,Q83620,thoroughfare
3,00015666,grande place,LOC_STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q234,Flandre,Q83620,thoroughfare
4,00015666,anvers,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Remise officielle de 40 drapeaux à la DM-ZB : ...,Marche organisée par la Milice thioise-Brigade...,750,NaN,Q31,BE,Belgique,Q12892,Anvers,Q56061,administrative territorial entity
10,00009589,maison du roi,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,Maison du roi - Bruxelles. [Sipho],Vues de Bruxelles : 1942-1944. - 124 photos,214,bruxelles (belgique)--photographies,Q31,BE,Belgique,Q239,Bruxelles,Q41176,building
11,00009589,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Maison du roi - Bruxelles. [Sipho],Vues de Bruxelles : 1942-1944. - 124 photos,214,bruxelles (belgique)--photographies,Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
12,00012338,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,La fête de Chants Flamands à Bruxelles le 31 a...,"Vlaams Nationaal Zangfeest à Bruxelles, 31/8/1...",831,NaN,Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
13,00012338,grand'place,LOC_STREET,http://pallas.cegesoma.be/pls/opac/opac.search...,La fête de Chants Flamands à Bruxelles le 31 a...,"Vlaams Nationaal Zangfeest à Bruxelles, 31/8/1...",831,NaN,Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare
14,00039619,paris,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Course cycliste Paris-Bruxelles. Groupe de pho...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,Q56061,administrative territorial entity


In [24]:
#Sélectionner les records qui contiennent moins de 3 loc_place (attention : brussels et bruxelles = 1 en fait. D'où, deux semble un bon compromis)
df_test = gb['spatial_key'].apply(lambda x: (x=='LOC_PLACE').sum()).reset_index(name='count_places')

In [25]:
df_test.head()

,beeldid,count_places
0,00000047,1
1,00000909,1
2,00002902,1
3,00003188,1
4,00005599,0


In [26]:
filtre = list(df_test[df_test['count_places'] < 3]['beeldid'])  
gb_filtered = data[data['beeldid'].isin(filtre)].groupby('beeldid')

In [27]:
gb_filtered.head()

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
10,00009589,maison du roi,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,Maison du roi - Bruxelles. [Sipho],Vues de Bruxelles : 1942-1944. - 124 photos,214,bruxelles (belgique)--photographies,Q31,BE,Belgique,Q239,Bruxelles,Q41176,building
11,00009589,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Maison du roi - Bruxelles. [Sipho],Vues de Bruxelles : 1942-1944. - 124 photos,214,bruxelles (belgique)--photographies,Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
12,00012338,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,La fête de Chants Flamands à Bruxelles le 31 a...,"Vlaams Nationaal Zangfeest à Bruxelles, 31/8/1...",831,NaN,Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
13,00012338,grand'place,LOC_STREET,http://pallas.cegesoma.be/pls/opac/opac.search...,La fête de Chants Flamands à Bruxelles le 31 a...,"Vlaams Nationaal Zangfeest à Bruxelles, 31/8/1...",831,NaN,Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare
14,00039619,paris,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Course cycliste Paris-Bruxelles. Groupe de pho...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,Q56061,administrative territorial entity
15,00039619,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Course cycliste Paris-Bruxelles. Groupe de pho...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,Q56061,administrative territorial entity
16,00000047,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Le bombardement aérien par les Anglais de la v...,"Bombardement de Bruxelles par les Allemands, 5...",5,campagne des 18 jours (1940)--bruxelles (belgi...,Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
17,00000047,rue de braie,LOC_STREET,http://pallas.cegesoma.be/pls/opac/opac.search...,Le bombardement aérien par les Anglais de la v...,"Bombardement de Bruxelles par les Allemands, 5...",5,campagne des 18 jours (1940)--bruxelles (belgi...,Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare
18,00038689,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"Bruxelles, le 5 septembre 1933. Des grands tra...","Travaux à Bruxelles et ailleurs, [...-1939]. -...",3181,belgique. ministere des travaux publics--bruxe...,Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
19,00038689,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"Bruxelles, le 5 septembre 1933. Des grands tra...","Travaux à Bruxelles et ailleurs, [...-1939]. -...",3181,belgique. ministere des travaux publics--bruxe...,Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity


In [28]:
#et maintenant appliquer la concatenation uniquement si le record ne contient pas plus de deux loc_place
gb = pd.DataFrame(gb_filtered['spatial_value'].apply(lambda x: "%s" % ' '.join(x))).reset_index()

In [29]:
gb.head()

,beeldid,spatial_value
0,00000047,bruxelles rue de braie
1,00000909,ostende
2,00002902,verviers
3,00003188,verviers
4,00005599,au petit sablon


In [30]:
#comment concaténer loc_name maintenant, et accéder à country_name ?
#en attendant de trouver une solution élégante, récréer un dataset contenant ces infos

gb = gb.merge(data[["beeldid", "loc_name", "country_name"]])

In [31]:
gb.head()

,beeldid,spatial_value,loc_name,country_name
0,00000047,bruxelles rue de braie,Bruxelles,Belgique
1,00000047,bruxelles rue de braie,Bruxelles,Belgique
2,00000909,ostende,NaN,France
3,00002902,verviers,NaN,NaN
4,00003188,verviers,NaN,Belgique


In [32]:
#on splitte les noms de pays multiples (n'existent plus)
#gb = tidy_split(gb.merge(data[["beeldid", "country_name", "loc_name"]]), column="country_name", sep=",", keep=False)

In [33]:
#on enlève les doublons
gb.drop_duplicates(subset=None, keep='first', inplace=True)

In [34]:
gb.head()

,beeldid,spatial_value,loc_name,country_name
0,00000047,bruxelles rue de braie,Bruxelles,Belgique
2,00000909,ostende,NaN,France
3,00002902,verviers,NaN,NaN
4,00003188,verviers,NaN,Belgique
5,00005599,au petit sablon,Bruxelles,Belgique


In [35]:
#on normalise les lieux et pays
gb['loc_name'] = gb['loc_name'].apply(lambda x: Fingerprinter(str(x)).get_fingerprint_nonsorted()).str.replace('nan',  '')
gb['country_name'] = gb['country_name'].apply(lambda x: Fingerprinter(str(x)).get_fingerprint_nonsorted()).str.replace('nan',  '')

In [36]:
# on concatène le tout en supprimant les doublons

gb['loc_concat'] = gb['spatial_value'].fillna('').map(
    str) + ' ' + gb['loc_name'].fillna('').map(str) + ' ' + gb['country_name'].fillna('').map(str)

gb['loc_concat'] = gb['loc_concat'].apply(
    lambda x: ' '.join(pd.unique(x.split())))

In [37]:
gb.head()

,beeldid,spatial_value,loc_name,country_name,loc_concat
0,00000047,bruxelles rue de braie,bruxelles,belgique,bruxelles rue de braie belgique
2,00000909,ostende,,france,ostende france
3,00002902,verviers,,,verviers
4,00003188,verviers,,belgique,verviers belgique
5,00005599,au petit sablon,bruxelles,belgique,au petit sablon bruxelles belgique


In [38]:
#appliquer get_nominatim. ATTENTION : ajouter le pays pour plus de précision

gb['nominatim'] = gb['loc_concat'].apply(get_nominatim)

D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=2004, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('10.45.5.44', 50610), raddr=('193.63.75.109', 443)>
  self._sock = None
D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=2772, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('10.45.5.44', 50612), raddr=('193.63.75.109', 443)>
  self._sock = None
D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=2656, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('10.45.5.44', 50614), raddr=('193.63.75.109', 443)>
  self._sock = None
D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=2880, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('10.45.5.44', 50616), raddr=('193.63.75.109', 443)>
  self._sock = None


In [39]:
#Attribuer un score de fiabilité aux résultats et choisir le meilleur s'il y en a un

def get_element(array, *args):
    """parse the first nominatim json."""
    for el in array:
        return dig(el, *args)
    
gb['nominatim_main_loc'] = np.vectorize(get_element)(gb['nominatim'], "display_name")
gb['nominatim_type_loc'] = np.vectorize(get_element)(gb['nominatim'], "type")
gb['nominatim_city_loc'] = np.vectorize(get_element)(gb['nominatim'], "address", "city")
gb['nominatim_country_loc'] = np.vectorize(get_element)(gb['nominatim'], "address", "country") 

In [47]:
gb

,beeldid,spatial_value,loc_name,country_name,loc_concat,nominatim,nominatim_main_loc,nominatim_type_loc,nominatim_city_loc,nominatim_country_loc,wikipedia_improved,wikidata
0,00000047,bruxelles rue de braie,bruxelles,belgique,bruxelles rue de braie belgique,[],None,None,None,None,Ville de Bruxelles (https://fr.wikipedia.org/w...,NaN
2,00000909,ostende,,france,ostende france,"[{'place_id': '48801331', 'licence': 'Data © O...","Ostende, Rue d'Ostende, Saint-Quentin, Aisne, ...",bus_stop,None,France,NaN,NaN
3,00002902,verviers,,,verviers,"[{'place_id': '178612889', 'licence': 'Data © ...","Verviers, Liège, Wallonie, 4800, Belgique",administrative,Verviers,Belgique,NaN,NaN
4,00003188,verviers,,belgique,verviers belgique,"[{'place_id': '178612889', 'licence': 'Data © ...","Verviers, Liège, Wallonie, 4800, Belgique",administrative,Verviers,Belgique,NaN,NaN
5,00005599,au petit sablon,bruxelles,belgique,au petit sablon bruxelles belgique,[],None,None,None,None,Sablon (Bruxelles) (https://fr.wikipedia.org/w...,NaN
6,00005817,bruxelles bourse,bruxelles,belgique,bruxelles bourse belgique,"[{'place_id': '49575968', 'licence': 'Data © O...","Oxfam Magasin du Monde, 137, Boulevard Anspach...",supermarket,Ville de Bruxelles,Belgique,NaN,NaN
8,00009589,maison du roi bruxelles,bruxelles,belgique,maison du roi bruxelles belgique,"[{'place_id': '74770722', 'licence': 'Data © O...","Maison du Roi, 29, Grand-Place, Quartier du Ce...",museum,Ville de Bruxelles,Belgique,NaN,NaN
10,00012338,bruxelles grand'place,bruxelles,belgique,bruxelles grand'place belgique,"[{'place_id': '68150353', 'licence': 'Data © O...","Hôtel de ville de Bruxelles, 8, Grand-Place, Q...",attraction,Ville de Bruxelles,Belgique,NaN,NaN
12,00016741,anvers,anvers,belgique,anvers belgique,"[{'place_id': '179011988', 'licence': 'Data © ...","Anvers, Flandre, Belgique",city,Anvers,Belgique,NaN,NaN
13,00019056,bruxelles,bruxelles,belgique,bruxelles belgique,"[{'place_id': '17872759', 'licence': 'Data © O...","Bruxelles, Bruxelles-Capitale, 1000, Belgique",city,Bruxelles,Belgique,NaN,NaN


In [48]:
#apply wikipedia only if nominatim failed
gb['wikipedia_improved'] = gb[pd.isnull(gb.nominatim_main_loc)]['loc_concat'].apply(lambda x: gsrsearch(x))

In [49]:
gb 

,beeldid,spatial_value,loc_name,country_name,loc_concat,nominatim,nominatim_main_loc,nominatim_type_loc,nominatim_city_loc,nominatim_country_loc,wikipedia_improved,wikidata
0,00000047,bruxelles rue de braie,bruxelles,belgique,bruxelles rue de braie belgique,[],None,None,None,None,Ville de Bruxelles (https://fr.wikipedia.org/w...,NaN
2,00000909,ostende,,france,ostende france,"[{'place_id': '48801331', 'licence': 'Data © O...","Ostende, Rue d'Ostende, Saint-Quentin, Aisne, ...",bus_stop,None,France,NaN,NaN
3,00002902,verviers,,,verviers,"[{'place_id': '178612889', 'licence': 'Data © ...","Verviers, Liège, Wallonie, 4800, Belgique",administrative,Verviers,Belgique,NaN,NaN
4,00003188,verviers,,belgique,verviers belgique,"[{'place_id': '178612889', 'licence': 'Data © ...","Verviers, Liège, Wallonie, 4800, Belgique",administrative,Verviers,Belgique,NaN,NaN
5,00005599,au petit sablon,bruxelles,belgique,au petit sablon bruxelles belgique,[],None,None,None,None,Sablon (Bruxelles) (https://fr.wikipedia.org/w...,NaN
6,00005817,bruxelles bourse,bruxelles,belgique,bruxelles bourse belgique,"[{'place_id': '49575968', 'licence': 'Data © O...","Oxfam Magasin du Monde, 137, Boulevard Anspach...",supermarket,Ville de Bruxelles,Belgique,NaN,NaN
8,00009589,maison du roi bruxelles,bruxelles,belgique,maison du roi bruxelles belgique,"[{'place_id': '74770722', 'licence': 'Data © O...","Maison du Roi, 29, Grand-Place, Quartier du Ce...",museum,Ville de Bruxelles,Belgique,NaN,NaN
10,00012338,bruxelles grand'place,bruxelles,belgique,bruxelles grand'place belgique,"[{'place_id': '68150353', 'licence': 'Data © O...","Hôtel de ville de Bruxelles, 8, Grand-Place, Q...",attraction,Ville de Bruxelles,Belgique,NaN,NaN
12,00016741,anvers,anvers,belgique,anvers belgique,"[{'place_id': '179011988', 'licence': 'Data © ...","Anvers, Flandre, Belgique",city,Anvers,Belgique,NaN,NaN
13,00019056,bruxelles,bruxelles,belgique,bruxelles belgique,"[{'place_id': '17872759', 'licence': 'Data © O...","Bruxelles, Bruxelles-Capitale, 1000, Belgique",city,Bruxelles,Belgique,NaN,NaN


In [42]:
#sélectionner les lignes où Nominatim et Wikipedia ont échoué
gb[(pd.isnull(gb.nominatim_main_loc)) & (pd.isnull(gb.wikipedia_improved))]

,beeldid,spatial_value,loc_name,country_name,loc_concat,nominatim,nominatim_main_loc,nominatim_type_loc,nominatim_city_loc,nominatim_country_loc,wikipedia_improved


In [43]:
#Chercher dans Wikidata les spatial_values individuelles là où tout a échoué. Pas oublier d'ungrouper.
gb['wikidata'] = gb[(pd.isnull(gb.nominatim_main_loc)) & (pd.isnull(gb.wikipedia_improved))]['spatial_value'].apply(lambda x: get_wikidata(x, 'Q618123'))